In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
sns.set_style('whitegrid')

import warnings
warnings.simplefilter('ignore')


In [2]:
df = pd.read_csv('/ihme/homes/rmbarber/covid-19/elastispliner/2020_06_07.08/model_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['location_id', 'date']).reset_index(drop=True)
df = df.loc[~df['Hospitalization rate'].isnull()]

first_date = df['date'] == df.groupby('location_id')['date'].transform(min)

df['est_daily_death_rate'] = np.diff(df['Predicted death rate (HFR)'], prepend=np.nan)
df.loc[first_date, 'est_daily_death_rate'] = df['Predicted death rate (HFR)']
df['est_cumul_deaths'] = df['Predicted death rate (HFR)'] * df['population']
df['est_daily_deaths'] = df['est_daily_death_rate'] * df['population']

df['daily_hospitalization_rate'] = np.diff(df['Hospitalization rate'], prepend=np.nan)
df.loc[first_date, 'daily_hospitalization_rate'] = df['Hospitalization rate']
df['cumul_hospitalizations'] = df['Hospitalization rate'] * df['population']
df['daily_hospitalizations'] = df['daily_hospitalization_rate'] * df['population']

df['cumul_hfr'] = df['cumul_hospitalizations'] / df['est_cumul_deaths']
df['daily_hfr'] = df['daily_hospitalizations'] / df['est_daily_deaths']

df = df[['location_id', 'location_name', 'date', 
         'est_cumul_deaths', 'est_daily_deaths', 
         'cumul_hospitalizations', 'daily_hospitalizations',
         'daily_hfr', 'cumul_hfr']]


In [3]:
with PdfPages('/ihme/homes/rmbarber/covid-19/hfr_2020_06_07.pdf') as pdf:
    for location_id in df['location_id'].unique():
        plot_df = df.loc[df['location_id'] == location_id].reset_index(drop=True)

        fig, ax = plt.subplots(1, 2, figsize=(16.5, 8.5))
        ax[0].scatter(plot_df['date'], plot_df['daily_hfr'], s=100)
        ax[0].plot(plot_df['date'], plot_df['daily_hfr'])
        ax[0].set_title('Daily', fontsize=14)
        ax[0].set_ylabel('HFR', fontsize=14)
        ax[0].set_xlabel('Date', fontsize=14)
        
        ax[1].scatter(plot_df['date'], plot_df['cumul_hfr'], s=100)
        ax[1].plot(plot_df['date'], plot_df['cumul_hfr'])
        ax[1].set_title('Cumulative', fontsize=14)
        ax[1].set_xlabel('Date', fontsize=14)

        # ax[0].set_yticks(fontsize=10)
        # ax[0].set_xticks(rotation=60, fontsize=10)
        # ax[1].set_yticks(fontsize=10)
        # ax[1].set_xticks(rotation=60, fontsize=10)
        
        fig.suptitle(plot_df['location_name'][0], y=1.0025, fontsize=20)
        fig.tight_layout()
        pdf.savefig()
        plt.close(fig)

df.to_csv(
    '/ihme/homes/rmbarber/covid-19/hfr_2020_06_07.csv', index=False
)


In [4]:
df

,location_id,location_name,date,est_cumul_deaths,est_daily_deaths,cumul_hospitalizations,daily_hospitalizations,daily_hfr,cumul_hfr
5236,89,Netherlands,2020-03-31,1034.065846,1034.065846,1230.0,1230.0,1.189479,1.189479
5237,89,Netherlands,2020-04-01,1176.298747,142.232902,1495.0,265.0,1.863141,1.270936
5238,89,Netherlands,2020-04-02,1347.275324,170.976576,1836.0,341.0,1.994425,1.362750
5239,89,Netherlands,2020-04-03,1495.817001,148.541678,2151.0,315.0,2.120617,1.438010
5240,89,Netherlands,2020-04-04,1644.385419,148.568417,2500.0,349.0,2.349086,1.520325
...,...,...,...,...,...,...,...,...,...
18540,573,Wyoming,2020-06-10,17.328324,0.049955,88.0,1.0,20.018009,5.078391
18541,573,Wyoming,2020-06-11,17.328324,0.000000,88.0,0.0,NaN,5.078391
18542,573,Wyoming,2020-06-12,17.328324,0.000000,88.0,0.0,NaN,5.078391
18543,573,Wyoming,2020-06-13,17.426975,0.098651,90.0,2.0,20.273522,5.164408
